https://en.wikipedia.org/wiki/Vincenty%27s_formulae

In [1]:
from numpy import pi, arctan, tan, sqrt, cos, sin, arctan2

def LLDist(lat1d,lon1d,lat2d,lon2d):
    # https://en.wikipedia.org/wiki/Vincenty%27s_formulae
    a=6378137.0
    f = 1/298.257223563
    b = (1-f)*a
    lat1r=lat1d*pi/180
    lon1r=lon1d*pi/180
    lat2r=lat2d*pi/180
    lon2r=lon2d*pi/180
    U1 = arctan((1-f)*tan(lat1r))
    U2 = arctan((1-f)*tan(lat2r))
    L = lon2r-lon1r
    lam = L

    for i in range(7):
        sinσ = sqrt((cos(U2)*sin(lam))**2+(cos(U1)*sin(U2)-sin(U1)*cos(U2)*cos(lam))**2)
        cosσ = sin(U1)*sin(U2)+cos(U1)*cos(U2)*cos(lam)
        σ = arctan2(sinσ,cosσ)
        sinα = cos(U1)*cos(U2)*sin(lam) / sinσ
        cos2α = 1 - sinα**2
        if cos2α == 0: cos2α = .000000000001
        cos2σm = cosσ - 2*sin(U1)*sin(U2) / cos2α
        C = f/16*cos2α*(4+f*(4-3*cos2α))
        lam = L+(1-C)*f*sinα*(σ+C*sinσ*(cos2σm+C*cosσ*(-1+2*cos2σm**2)))

    u2 = cos2α*(a**2/b**2-1)
    k1 = (sqrt(1+u2)-1)/(sqrt(1+u2)+1)
    A = (1+.25*k1**2)/(1-k1)
    B = k1*(1-3/8*k1**2)
    Δσ = B*sinσ*(cos2σm+B/4*(cosσ*(-1+2*cos2σm**2)-B/6*cos2σm*(-3+4*sinσ**2)*(-3+4*cos2σm**2)))
    dist = b*A*(σ-Δσ)

    lazr = arctan2(sin(lon2r-lon1r)*cos(lat2r),cos(lat1r)*sin(lat2r)-sin(lat1r)*cos(lat2r)*cos(lon2r-lon1r))
    lazd = (lazr*180/pi+360)%360
    bazr = arctan2(sin(lon1r-lon2r)*cos(lat1r),cos(lat2r)*sin(lat1r)-sin(lat2r)*cos(lat1r)*cos(lon1r-lon2r))
    bazd = (bazr*180/pi+360)%360
    iazd = (bazd+180)%360

    return [dist,lazd,bazd,iazd]



In [2]:
LLDist(30,30,-40,-40)

[10592890.730628857, 226.30950192831196, 54.83087023279677, 234.83087023279677]

In [3]:
from numpy import pi, arctan, tan, sqrt, cos, sin, arctan2

def vPolar(lat1d,lon1d,lazd,dists):
    # https://en.wikipedia.org/wiki/Vincenty%27s_formulae
    a=6378137.0
    f = 1/298.257223563
    b = (1-f)*a
 
    lat1r=lat1d*pi/180
    lon1r=lon1d*pi/180
    lazr = lazd*pi/180
    U1 = arctan((1-f)*tan(lat1r))
    sig1 = arctan2(tan(U1),cos(lazr))
    sinα = cos(U1)*sin(lazr)
    u2 = (1-sinα**2)*(a**2/b**2-1)
    k1 = (sqrt(1+u2)-1)/(sqrt(1+u2)+1)
    A = (1+.25*k1**2)/(1-k1)
    B = k1*(1-3/8*k1**2)
    sig = dists/b/A

    for i in range(4):
        tsm = 2*sig1+sig
        dsig = B*sin(sig)*(cos(tsm)+B/4*(cos(sig)*(-1+2*cos(tsm))-B/6*cos(tsm)*(-3+4*sin(sig)**2)*(-3+4*cos(tsm)**2)))
        sig = dists/b/A+dsig

    lat2r = arctan2(sin(U1)*cos(sig)+cos(U1)*sin(sig)*cos(lazr),(1-f)*sqrt(sinα**2+(sin(U1)*sin(sig)-cos(U1)*cos(sig)*cos(lazr))**2))
    lam = arctan2(sin(sig)*sin(lazr),cos(U1)*cos(sig)-sin(U1)*sin(sig)*cos(lazr))
    C = f/16*(1-sinα**2)*(4+f*(4-3*(1-sinα**2)))
    L = lam-(1-C)*f*sinα*(sig+C*sin(sig)*(cos(tsm)+C*cos(sig)*(-1+2*cos(tsm)**2)))
    lon2r = L+lon1r
    lat2d = lat2r*180/pi
    lon2d = lon2r*180/pi
    more = LLDist(lat1d,lon1d,lat2d,lon2d)
    iazd = more[3]
    return [lat2d,lon2d,lazd,iazd]

In [4]:
vPolar(30,30,250,1000000)

[26.576021889618417, 20.563612677272232, 250, 245.41326717553181]

In [5]:
# I am Creating a reverse polar method

from numpy import sqrt,pi,cos,sin,tan,arctan,arctan2,arcsin

def revVpolar(lat2d,lon2d,lazd,distm):
    rlazl = (lazd+180)%360
    taz = rlazl
    for i in range(20):
        temp = vPolar(lat2d,lon2d,taz,distm)
        #print(temp)
        taz = taz - (temp[3]-rlazl)
        

    lat1d = temp[0]
    lon1d = temp[1]
    liazr = (taz +180)%360

    return [lat1d,lon1d, lazd, liazr]



In [6]:
revVpolar(30,30,65,1000000)

[26.514588513178506, 20.595011333492717, 65, 69.56900805165316]